In [1]:
import pandas as pd

In [7]:
df_train = pd.read_csv('/content/Data_train.csv')
df_test = pd.read_csv('/content/Data_test.csv')

In [8]:
df_train.shape

(700, 8)

In [9]:
df_train.head()

,type,group,education,meal,preparation course,score-1,score-2,score-3
0,wild,group B,some high school,standard,completed,63,67,67
1,domestic,group C,bachelor's degree,standard,none,67,69,75
2,domestic,group C,some college,standard,completed,69,90,88
3,domestic,group B,some high school,standard,none,62,64,66
4,wild,group E,some college,reduced,none,93,90,83


In [12]:
df_train.isna().sum()

type                   0
group                 20
education              0
meal                   0
preparation course     0
score-1                0
score-2                0
score-3                0
dtype: int64

In [15]:
df_test.isna().sum()

type                   0
group                 12
education              0
meal                   0
preparation course     0
dtype: int64

In [14]:
df_train.dtypes

type                  object
group                 object
education             object
meal                  object
preparation course    object
score-1                int64
score-2                int64
score-3                int64
dtype: object

In [20]:
df_train = df_train.fillna({'group': 'None'})
df_test = df_test.fillna({'group': 'None'})

In [21]:
df_train.isna().sum()

type                  0
group                 0
education             0
meal                  0
preparation course    0
score-1               0
score-2               0
score-3               0
dtype: int64

In [23]:
df_test.isna().all().all()

False

In [27]:
df_train['Pass'] = ((df_train['score-1'] > 50) & (df_train['score-2'] > 50) & (df_train['score-3'] > 50))* 1

In [30]:
df_train['Pass'].value_counts()

1    555
0    145
Name: Pass, dtype: int64

In [35]:
wild_d = len(df_train[(df_train['Pass'] == 1) & (df_train['type'] == 'wild')])/len(df_train[df_train['type'] == 'wild'])
wild_d

0.7813411078717201

In [42]:
domestic_d = len(df_train[(df_train['Pass'] == 1) & (df_train['type'] == 'domestic')])/len(df_train[df_train['type'] == 'domestic'])
domestic_d

0.803921568627451

In [45]:
df_train['preparation course'].value_counts()

none         442
completed    258
Name: preparation course, dtype: int64

In [43]:
domestic_d - wild_d

0.022580460755730947

In [46]:
len(df_train[(df_train['Pass'] == 1) & (df_train['type'] == "wild") & (df_train['preparation course'] == "none")])

152

In [48]:
df_train['score-1'].median()

66.0

In [50]:
df_train[(df_train['type'] == "domestic") & (df_train['preparation course'] == "none")]['score-3']

,type,group,education,meal,preparation course,score-1,score-2,score-3,Pass
1,domestic,group C,bachelor's degree,standard,none,67,69,75,1
3,domestic,group B,some high school,standard,none,62,64,66,1
7,domestic,group B,some high school,standard,none,66,69,68,1
10,domestic,group A,high school,standard,none,55,73,73,1
11,domestic,group D,high school,reduced,none,49,57,52,0
...,...,...,...,...,...,...,...,...,...
689,domestic,group D,master's degree,standard,none,87,100,100,1
690,domestic,group D,master's degree,standard,none,55,64,70,1
691,domestic,group D,some high school,standard,none,51,63,61,1
694,domestic,group E,some high school,reduced,none,38,49,45,0


In [51]:
import numpy as np

In [52]:
q3, q1 = np.percentile (df_train[(df_train['type'] == "domestic") & (df_train['preparation course'] == "none")]['score-3'], [75 ,25])

In [53]:
q3 - q1

20.0

In [74]:
from sklearn.preprocessing import OneHotEncoder

In [75]:
from sklearn.compose import ColumnTransformer

In [77]:
categorical = df_train.columns[:5]
categorical

Index(['type', 'group', 'education', 'meal', 'preparation course'], dtype='object')

In [176]:
df_train_new = df_train[categorical]

In [174]:
column_transformer = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown="ignore"), categorical)
], remainder='passthrough')


In [175]:
old_num = df_train_new.shape[1]
old_num

6

In [87]:
df_train_encoded = column_transformer.fit_transform(df_train_new)

In [88]:
df_test_encoded = column_transformer.transform(df_test)

In [89]:
new_num = df_train_encoded.shape[1]
new_num

18

In [84]:
22 - 9

13

In [90]:
X_train, y_train = df_train_encoded, df_train['Pass']

In [93]:
X_train.shape

(700, 18)

In [92]:
y_train.shape

(700,)

In [94]:
from sklearn.tree import DecisionTreeClassifier

In [95]:
from sklearn.model_selection import GridSearchCV

In [111]:
from sklearn.metrics import roc_auc_score

In [104]:
# grid_searcher = GridSearchCV(estimator=DecisionTreeClassifier(),
#              param_grid={'max_depth': [5],
#                          'criterion': ['entropy']},
#              cv = 3,
#              scoring = roc_auc_score)


In [119]:
clf = DecisionTreeClassifier(max_depth = 5,criterion = 'entropy' )

In [118]:
from sklearn.model_selection import cross_val_score

In [120]:
scores = cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc')

In [121]:
scores

array([0.66569222, 0.69420045, 0.62387387])

In [122]:
scores.mean()

0.6612555157198013

In [125]:
range(2, 21, 2)

range(2, 21, 2)

In [128]:
grid_searcher = GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [i for i in range(2, 21, 2)]},
             cv = 3,
             scoring = 'roc_auc')

In [129]:
grid_searcher.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]},
             scoring='roc_auc')

In [130]:
grid_searcher.best_params_

{'max_depth': 2}

In [132]:
df_train_new

,type,group,education,meal,preparation course
0,wild,group B,some high school,standard,completed
1,domestic,group C,bachelor's degree,standard,none
2,domestic,group C,some college,standard,completed
3,domestic,group B,some high school,standard,none
4,wild,group E,some college,reduced,none
...,...,...,...,...,...
695,wild,group C,some high school,standard,completed
696,domestic,group B,high school,reduced,none
697,wild,group E,some college,standard,completed
698,wild,group E,associate's degree,standard,none


In [422]:
df_train_new['cat_bio'] = df_train_new.apply(lambda row: tuple((row['type'], row['group'])), axis=1)

<ipython-input-422-dd511573d741>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_new['cat_bio'] = df_train_new.apply(lambda row: tuple((row['type'], row['group'])), axis=1)


In [417]:
df_test_new = df_test.copy()

In [423]:
df_test_new['cat_bio'] = df_test_new.apply(lambda row: tuple((row['type'], row['group'])), axis=1)

In [424]:
df_test_new['cat_bio'] = str(df_test_new['type']) + str(df_test_new.group)

In [425]:
df_train_new['cat_bio'] = f"{df_train_new['type']}, {df_train_new.group} )"

<ipython-input-425-ff9ac58ab752>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_new['cat_bio'] = f"{df_train_new['type']}, {df_train_new.group} )"


In [426]:
ohe = OneHotEncoder(handle_unknown="ignore")

In [427]:
train_encoded = ohe.fit_transform(df_train_new)

In [428]:
test_encoded = ohe.transform(df_test_new)

In [429]:
train_encoded.shape

(700, 19)

In [430]:
X, y = train_encoded, df_train['Pass']

In [431]:
clf = DecisionTreeClassifier(
    max_depth = 5,
    criterion = 'entropy')

clf.fit(X, y)

DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [432]:
roc_auc_score(y, clf.predict(X))

0.5753960857409133

In [365]:
from sklearn.ensemble import RandomForestClassifier

In [383]:
clf = RandomForestClassifier(n_estimators=100, max_depth = 3)

In [384]:
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=3)

In [385]:
predictions = clf.predict(df_test_encoded)

In [386]:
roc_auc_score(y_train, clf.predict(X_train))

0.5137931034482759

In [370]:
with open('/content/result.txt', 'w') as f:
    for i in range(len(predictions)):
      if i <len(predictions )-1:
          f.write(str(predictions[i]) + '\n')
      else:
          f.write(str(predictions[i]))

In [218]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 3.0 MB/s eta 0:00:00


In [336]:
from catboost import CatBoostClassifier

In [362]:
clf = CatBoostClassifier(iterations=20, learning_rate=0.5, depth=8)
clf.fit(X_train, y_train)

0:	learn: 0.5784383	total: 747us	remaining: 14.2ms
1:	learn: 0.5051125	total: 1.37ms	remaining: 12.4ms
2:	learn: 0.4669327	total: 2.27ms	remaining: 12.9ms
3:	learn: 0.4471176	total: 3.14ms	remaining: 12.6ms
4:	learn: 0.4320789	total: 3.56ms	remaining: 10.7ms
5:	learn: 0.4212150	total: 4.35ms	remaining: 10.2ms
6:	learn: 0.4113055	total: 5.19ms	remaining: 9.63ms
7:	learn: 0.4029027	total: 6.02ms	remaining: 9.03ms
8:	learn: 0.3959379	total: 6.86ms	remaining: 8.38ms
9:	learn: 0.3872518	total: 7.67ms	remaining: 7.67ms
10:	learn: 0.3803558	total: 8.47ms	remaining: 6.93ms
11:	learn: 0.3781253	total: 9.07ms	remaining: 6.05ms
12:	learn: 0.3744552	total: 10.1ms	remaining: 5.44ms
13:	learn: 0.3693304	total: 11ms	remaining: 4.73ms
14:	learn: 0.3642956	total: 11.9ms	remaining: 3.96ms
15:	learn: 0.3610473	total: 12.7ms	remaining: 3.17ms
16:	learn: 0.3562033	total: 13.5ms	remaining: 2.38ms
17:	learn: 0.3530554	total: 14.3ms	remaining: 1.59ms
18:	learn: 0.3502038	total: 15.2ms	remaining: 798us
19:	lea

In [363]:
predictions = clf.predict(df_test_encoded)

In [364]:
roc_auc_score(y_train, clf.predict(X_train))

0.6579993786890338

In [355]:
with open('/content/result.txt', 'w') as f:
    for i in range(len(predictions)):
      if i <len(predictions )-1:
          f.write(str(predictions[i]) + '\n')
      else:
          f.write(str(predictions[i]))

In [387]:
from sklearn.linear_model import LogisticRegression

In [396]:
clf = LogisticRegression(max_iter = 1000)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [397]:
predictions = clf.predict(df_test_encoded)

In [398]:
roc_auc_score(y_train, clf.predict(X_train))

0.5455420938179559

In [399]:
with open('/content/result.txt', 'w') as f:
    for i in range(len(predictions)):
      if i <len(predictions )-1:
          f.write(str(predictions[i]) + '\n')
      else:
          f.write(str(predictions[i]))